In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Dataset/train.csv")

In [3]:
df.head(8)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0


In [4]:
df['text']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20800, dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [7]:
df=df.dropna()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [9]:
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.11.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
#vocabulary size
voc_size=10000

In [19]:
messages=X.copy()

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zeel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [24]:
from tqdm import tqdm

In [25]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in tqdm(range(0,len(messages))):
#     print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
#     print(review)
    review = review.split()
#     print(review)
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

100%|███████████████████████████████████████████████████████████████████████████| 18285/18285 [00:41<00:00, 440.32it/s]


In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
len(corpus)

18285

In [28]:
onehot_representation = [one_hot(words,voc_size) for words in corpus]
onehot_representation

[[7214, 9666, 3433, 3436, 5879, 2531, 6742, 6898, 9646, 7867],
 [6089, 9255, 4166, 810, 4533, 1722, 5069],
 [2358, 8580, 1786, 4876],
 [1656, 4223, 1040, 6592, 3686, 8109],
 [1086, 4533, 1419, 5670, 639, 3722, 4533, 7078, 3442, 6782],
 [8741,
  7767,
  4890,
  685,
  5410,
  1955,
  8162,
  5172,
  2199,
  5640,
  9246,
  3700,
  3570,
  6173,
  5069],
 [2852, 3000, 6930, 9291, 9830, 8021, 1783, 739, 7758, 4708, 6610],
 [1952, 5522, 1086, 6419, 6475, 2362, 1955, 6583, 7758, 4708, 6610],
 [1345, 9729, 9033, 4606, 7477, 9347, 2736, 2607, 1955, 4164],
 [6373, 353, 6283, 1973, 1100, 7949, 8737, 6649],
 [1038, 5457, 8769, 6078, 7462, 1505, 2106, 6808, 169, 823, 265],
 [6592, 7854, 5879, 9347, 1955, 6475],
 [3273, 6554, 2152, 3024, 8657, 4618, 6906, 1374, 222],
 [7160, 3121, 5200, 6932, 7019, 2388, 7483, 7758, 4708, 6610],
 [1680, 9077, 371, 7263, 1133, 7758, 4708, 6610],
 [5368, 4462, 9675, 513, 9647, 9255, 5653, 8720, 5250, 3230],
 [3115, 9294, 9255],
 [3796, 4716, 2247, 8164, 1955, 1491, 

In [29]:
sent_length=100
embedded_docs=pad_sequences(onehot_representation,padding='pre',maxlen=sent_length)

In [30]:
embedded_docs

array([[   0,    0,    0, ..., 6898, 9646, 7867],
       [   0,    0,    0, ..., 4533, 1722, 5069],
       [   0,    0,    0, ..., 8580, 1786, 4876],
       ...,
       [   0,    0,    0, ..., 7758, 4708, 6610],
       [   0,    0,    0, ...,   53, 9556, 3060],
       [   0,    0,    0, ..., 3417,  407, 2712]])

In [31]:
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 100, 128)          117248    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,168,769
Trainable params: 1,168,769
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 100), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [37]:
len(X_train),len(X_test)

(12250, 6035)

In [38]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=256)

Epoch 1/10
48/48 [==============================] - 59s 1s/step - loss: 0.5068 - accuracy: 0.7362 - val_loss: 0.2726 - val_accuracy: 0.8848
Epoch 2/10
48/48 [==============================] - 64s 1s/step - loss: 0.1713 - accuracy: 0.9324 - val_loss: 0.1896 - val_accuracy: 0.9201
Epoch 3/10
48/48 [==============================] - 68s 1s/step - loss: 0.0959 - accuracy: 0.9654 - val_loss: 0.2292 - val_accuracy: 0.9188
Epoch 4/10
48/48 [==============================] - 69s 1s/step - loss: 0.0618 - accuracy: 0.9799 - val_loss: 0.2573 - val_accuracy: 0.9185
Epoch 5/10
48/48 [==============================] - 75s 2s/step - loss: 0.0372 - accuracy: 0.9883 - val_loss: 0.3291 - val_accuracy: 0.9128
Epoch 6/10
48/48 [==============================] - 76s 2s/step - loss: 0.0266 - accuracy: 0.9923 - val_loss: 0.3625 - val_accuracy: 0.9122
Epoch 7/10
48/48 [==============================] - 70s 1s/step - loss: 0.0165 - accuracy: 0.9950 - val_loss: 0.4050 - val_accuracy: 0.9112
Epoch 8/10
48/48 [==

In [39]:
# predict_x=model.predict(X_test) 
# y_pred=np.argmax(predict_x,axis=1)

In [40]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 4s 22ms/step


In [41]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [42]:
y_test

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [43]:
from sklearn.metrics import confusion_matrix

In [44]:
confusion_matrix(y_test,y_pred)

array([[3135,  284],
       [ 251, 2365]], dtype=int64)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9113504556752279

In [47]:
from sklearn.metrics import classification_report

In [49]:
print(classification_report(y_test, y_pred, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.93      0.92      0.92      3419
        Real       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

